# 🚀 GraphQL to MCP Starterkit

**Do you want to use your GraphQL APIs with agents? Let's give it a shot.**

## 📋 Goals

### The Basics
1. **GraphQL ↔ AI Concepts**: How schemas become tool contracts
2. **Safety First**: Queries vs mutations in AI context  
3. **Hands-On Examples**: Real GraphQL APIs → MCP

## Connect your GQL APIs to real AI apps
4. **AI Client Integration**: Claude & Cursor setup
5. **Schema Design**: Writing AI-friendly documentation

## Beyond the homelab setup
6. **Production Ready**: Authentication, governance, monitoring
7. **Toolprint Integration**: Smart tool orchestration

## 🎯 Prerequisites

- Basic GraphQL knowledge (queries, mutations, schemas)
- Command line comfort
- Node.js installed
- An existing GraphQL API (or we'll use examples)


---


## 🧠 Chapter 1: GraphQL Meets AI - The Conceptual Bridge

### Why GraphQL + AI = Perfect Match

GraphQL APIs are uniquely suited for AI tool integration because they are:

- **🔍 Self-Documenting**: Schemas contain type information and descriptions
- **🎯 Strongly Typed**: Clear parameter validation and return types  
- **🔎 Introspectable**: APIs can describe themselves programmatically
- **⚡ Efficient**: Single endpoint, precise field selection

### The Challenge: Raw GraphQL ≠ AI-Ready Tools

However, GraphQL APIs aren't immediately AI-accessible:

```graphql
# This GraphQL query...
query GetUser($id: ID!) {
  user(id: $id) {
    id
    name
    posts {
      title
      author {  # 🚨 Circular reference!
        name
      }
    }
  }
}
```

**AI agents struggle with:**
- Writing complex field selections
- Handling circular references  
- Understanding parameter requirements
- Knowing which tools exist


### The MCP Solution: Universal Tool Protocol

**Model Context Protocol (MCP)** bridges this gap by providing a standardized way for AI tools to communicate with data sources and APIs.

The solution requires something that can:
1. **Introspect the GraphQL schema** to understand available operations
2. **Generate individual tools** for each query and mutation
3. **Handle field selection automatically** to avoid GraphQL validation errors
4. **Provide proper parameter validation** before execution


<img alt="Basic architecture of a MCP <> GraphQL Proxy" src="./assets/mcp-graphql-proxy.png" width="50%" height="50%">

### 🍱 Some Food for Thought

Before we dive into implementation, there are important caveats to consider:

#### 1. **Mutation Safety**: Should mutations be exposed as tools?
```graphql
# This becomes a tool that could delete data!
type Mutation {
  deleteUser(id: ID!): Boolean
  chargeCard(amount: Float!): PaymentResult  
}
```
**Question**: Should AI agents have unrestricted access to write operations?

#### 2. **Data Shape**: How much data should we return?
```graphql
# Minimal - fast, fits in context
{ id, name }

# Expansive - comprehensive, but large
{ id, name, email, posts { title, content, comments { author, text } } }
```
**Consideration**: Context window limits vs completeness

#### 3. **Query Composition**: Some queries only make sense together
```graphql
# Step 1: Get user ID
user(username: "john") { id }

# Step 2: Get user's posts (needs ID from step 1)  
posts(authorId: "user-123") { title }
```
#### 4. **Challenge**: Individual tools may not provide complete workflows

## 🔬 Chapter 3: DEMO - make a real GraphQL service an MCP

Option 1 - **connect to toolprint graphql api without authentication**
_Note that this may take a few seconds to load up because it's running on Strapi's free tier_

This will be your environment variable for the next step:
```
export GRAPHQL_ENDPOINT=https://patient-song-383e3762d2.strapiapp.com/graphql
```

In [ ]:
GRAPHQL_ENDPOINT=https://patient-song-383e3762d2.strapiapp.com/graphql \
npx -y @toolprint/mcp-graphql-forge --transport http --port 3001

Option 2 - **connect to the github graphql API**
1. Grab your existing Github Personal Access Token (! Ensure that it only has READ perms for safety) OR [follow this guide](https://docs.github.com/en/rest/authentication/authenticating-to-the-rest-api?apiVersion=2022-11-28) to create a new one. 

These will be your environment variables for the next step:
```
export GRAPHQL_ENDPOINT=https://api.github.com/graphql
export GRAPHQL_AUTH_HEADER=Bearer ${GH_TOKEN}
```

In [ ]:
GRAPHQL_ENDPOINT=https://api.github.com/graphql \
GRAPHQL_AUTH_HEADER=Bearer ${GH_TOKEN} \
npx -y @toolprint/mcp-graphql-forge --transport http --port 3001

Now we'll inspect our tools using [mcp inspector](https://github.com/modelcontextprotocol/inspector)
When we launch the inspector, grab the URL outputted in the terminal and open it up. Then, connect to our launched MCP <> GraphQL Proxy on the connection menu like so:

<img alt="Basic architecture of a MCP <> GraphQL Proxy" src="./assets/connect-via-inspector.png" width="30%" height="30%">

**Note that our mcp <> graphql proxy is using the Streamable HTTP transport via the --transport http flag**

## Exploring Our Tools in MCP Inspector

Now that we have our MCP server running, let's explore the tools it generated using the MCP Inspector. The inspector provides a visual interface to:
1. Browse all available tools
2. Understand tool parameters and requirements
3. Test tool execution in real-time

### Connecting to the Inspector

1. Open the MCP Inspector URL shown in your terminal
2. Click "Connect to MCP Server" in the top right
3. Enter the connection URL: `http://localhost:3001/mcp`
4. Click "Connect"

You should now see a list of all tools generated from your GraphQL schema!

### Understanding Tool Structure

Let's examine some tools in detail:

#### Query Tools (Safe Read Operations)
- `query_articles` - List content articles
- `query_categories` - Get content categories
- `query_users` - Retrieve user information

Notice how each tool shows:
- 📝 **Description** - From your GraphQL schema documentation
- 🔍 **Parameters** - Required (marked with *) vs Optional
- 📊 **Response Type** - Expected return data structure

### Try These Examples

1. **Simple Query (Optional Parameters)**
   - Tool: `query_articles`
   - Parameters:
   ```json
   {
     "pagination": {
       "limit": 5
     }
   }
   ```
   - Notice: `pagination` is optional, but when provided must follow the schema

2. **Query with Required Parameters**
   - Tool: `query_article`
   - Parameters:
   ```json
   {
     "id": "1"  // ! Use an id value from the previous example.
   }
   ```
   - Try: Remove the `id` and see validation prevent execution

### Understanding the Magic

This clear parameter structure exists because `mcp-graphql-forge` has:
1. **Introspected** your GraphQL schema
2. **Analyzed** field types and nullability
3. **Generated** JSON Schema validation
4. **Created** individual tools with precise contracts

No more guessing about:
- Which fields are required
- What types parameters should be
- How to structure nested objects
- What the tool actually does

The inspector makes all this visible and testable in a user-friendly interface!

## 🤖 Chapter 4: Connecting to AI Clients (Claude/Cursor)

Now that we've tested our GraphQL-to-MCP bridge, let's connect it to actual AI clients where the real magic happens.

### The Transport Switch: HTTP → Stdio

**Important**: AI clients like Claude and Cursor use **stdio transport**, not Streamable HTTP.

- **HTTP Transport** = Use with real deployed AI applications / Inspector
- **Stdio Transport** = For hyperlocal AI applications

### Optional Step: Configure for Claude Desktop

Create or edit your `claude_desktop_config.json` file:

**Location**:
- **macOS**: `~/Library/Application Support/Claude/claude_desktop_config.json`
- **Windows**: `%APPDATA%\Claude\claude_desktop_config.json`


In [ ]:
{
  "mcpServers": {
    "mcp-graphql-forge": {
      "command": "npx",
      "args": [
        "-y",
        "@toolprint/mcp-graphql-forge"
      ],
      "env": {
        "GRAPHQL_ENDPOINT": "https://patient-song-383e3762d2.strapiapp.com/graphql"
      }
    }
  }
}


### Optional Step 2: Configure for Cursor

Create or edit your `mcp.json` - Settings > Cursor Settings > Tools & Integrations > MCP Tools


In [ ]:
{
  "mcpServers": {
    "mcp-graphql-forge": {
      "command": "npx",
      "args": [
        "-y", 
        "@toolprint/mcp-graphql-forge"
      ],
      "env": {
        "GRAPHQL_ENDPOINT": "https://patient-song-383e3762d2.strapiapp.com/graphql"
      }
    }
  }
}


### Step 3: Test the Integration

#### For Claude Desktop:
1. **Save the config file** and restart Claude Desktop
2. **Look for the 🔌 icon** in the Claude interface
3. **Start a conversation**: "What articles are available in the CMS?"
4. **Watch Claude use the tools** automatically!

#### For Cursor:
1. **Save the mcp.json file** in your project root
2. Toggle the MCP server until the icon shows greens and shows tools (you may notice that it has tool overload!).
3. **Ask about your CMS**: "Show me the latest articles from the CMS"
4. **See Cursor query your GraphQL API** through the generated tools

### 🎯 Test Prompts to Try

#### Safe Exploration (Queries Only)
- "What content types are available in the CMS?"
- "Show me the 5 most recent articles"
- "List all categories and their article counts"
- "Find articles containing 'tutorial' in the title"

#### Advanced Analysis  
- "Analyze the content strategy - which categories have the most articles?"
- "Create a summary of all article titles and their publication dates"
- "What are the top authors by article count?"

### 🛡️ Safety Note

Remember: The tools generated include both **queries** (safe) and **mutations** (potentially destructive). For production use, consider:
- Starting with query-only mode
- Adding authentication requirements
- Implementing audit logging for mutations


## 🤔 Did Your AI Assistant Seem Lost?

If you tried the examples above with Claude or Cursor, you might have noticed something interesting:
- Did the AI seem to hallucinate tools that don't exist?
- Did it struggle to find the right tool for your task?
- Did it get overwhelmed by the number of available tools?

**You're not alone!** This is a common challenge when exposing large GraphQL APIs as MCP tools.

### The Tool Discovery Challenge

Consider what happened when we connected to GitHub's GraphQL API:
- 100+ generated tools
- Complex nested parameters
- Similar-sounding tool names
- Tools that work better together

Even for a sophisticated AI like Claude or Cursor, this creates a **cognitive overload**. The AI needs to:
1. Search through a massive tool list
2. Understand complex parameter requirements
3. Figure out tool relationships
4. Plan multi-step workflows

### Enter Toolprint: Smart Tool Discovery

This is exactly why we built [Toolprint](https://toolprint.ai) - to make large tool collections manageable through:

#### 1. Semantic Tool Search
Instead of having LLMs have an overflowing context of 100s of tools, they can search and find the BEST tools given their goals using
the toolprint's tool_search functionality.
```
"Find repository information" → [github::query_repository, 9 more tools...]
"Create an issue on linear" → [linear::create_issue, ...]
```

Using these tool references, the LLM can call toolprint to execute the chosen tools using a referential ID and the correct
parameters given the shape of the tools offered by tool search.

#### 2. Baking in relationships between tools & building real workflows!
Toolprint also offers up the concept of a "toolprint" which is a workflow that can be crafted in plain English 
with tools across all your MCPs. Passing a toolprint to an LLM gives it clear instructions on how to execute
a workflow without having to write a single line of code.

The best part is that if you connect your IDE to the `toolprint mcp` server, it can author it all for you with the right references!

```yaml
# Example -

goal: Track, analyze, and action changes in the MCP protocol repository by monitoring PRs, issues, and discussions, then creating linear tickets for required implementation work.

instructions: |
  1. Fetch recent activity from the MCP protocol repository:
     - Get merged PRs from the last 2 weeks
     - Get active issues with the 'spec' label
     - Get discussions tagged with 'protocol'
  
  2. For each PR and issue:
     - Extract key comments and decisions
     - Note any breaking changes
     - Identify implementation requirements
  
  3. Generate a summary report:
     - Group changes by protocol component
     - Highlight breaking changes
     - List pending decisions
  
  4. For each implementation requirement:
     - Create a Linear ticket with appropriate context
     - Link back to source PR/issue
     - Tag with 'mcp-protocol-update'
     - Set priority based on breaking change status

tools:
  - ref:
      name: github_list_prs
      usage_hints: |
        # Use to fetch recent PRs
        - Filter by merged status and date range
        - Sort by update date descending
        - Include labels and review status
  
  - ref:
      name: github_list_issues
      usage_hints: |
        # Use to fetch active issues
        - Filter by 'spec' label
        - Include comments and reactions
        - Sort by activity
  
  - ref:
      name: github_get_discussions
      usage_hints: |
        # Use to fetch relevant discussions
        - Filter by 'protocol' category
        - Include all comments
        - Sort by last activity
  
  - ref:
      name: linear_create_ticket
      usage_hints: |
        # Use to create implementation tickets
        - Set title with clear context
        - Include links to source PR/issue
        - Add 'mcp-protocol-update' label
        - Set priority field
        - Add implementation requirements to description
```

#### 3. Use the tool better next time
Toolprint offers meta tools to have LLMs (or humans) reflect on how to use tools better beyond their meager descriptions!

Have the AI reflect on tool use and add annotations on the tool itself so that the next time the AI needs to use the tool, it can use the annotations to understand the tool better.

Ex. The Linear `list_issues` tool on its own does not instruct an LLM that it needs to provide a REAL project identifier so an LLM will hallucinate values. Solution: add a tool annotation: `ai_note: find the available teams using list_teams in order to populate the team value in this tool`


### Getting Started with Toolprint

1. **Install the Toolprint CLI**:
```bash
brew install toolprint/tap/toolprint
```

2. **Join our free sandbox to see how it all works across 8 MCPs and over 70 tools**:
```bash
toolprint
```

Visit [toolprint.ai](https://toolprint.ai) to learn more about making your GraphQL tools more discoverable and usable for AI agents!
